In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats

from gm import gm
from gm_Prep import gm_Prep

In [ ]:
# np.random.seed(1337)

In [ ]:
# SBML model we want to import
sbml_file = 'SPARCED_Brep.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4]
# Directory to which the generated model code is written
model_output_dir = model_name

kGsRead = pd.read_csv('OmicsData.txt',header=0,index_col=0,sep="\t")
gExp_mpc = np.float64(kGsRead.values[:,0])
mExp_mpc = np.float64(kGsRead.values[:,1])
kGin = np.float64(kGsRead.values[:,2])
kGac = np.float64(kGsRead.values[:,3])
kTCleak = np.float64(kGsRead.values[:,4])
kTCmaxs = np.float64(kGsRead.values[:,5])
kTCd = np.float64(kGsRead.values[:,6])
kmRNAs = kGsRead.values[:,7]

# Read-in the activators matrix and assign concentrations of activators
TAsRead = pd.read_csv('TAs.csv',header=0,index_col=0)
TAs0 = np.float64(TAsRead.values)
# Read-in the repressors matrix and assign concentrations of repressors
TRsRead = pd.read_csv('TRs.csv',header=0,index_col=0)
TRs0 = np.float64(TRsRead.values)

In [ ]:
# # Etop With GFs
flagD = 0
th1 = 24 # Time to simulate wihout EGF, Insulin, or Etoposide
th2 = 72 # Time to simulate with EGF + Insulin
th3 = 72 # Time to simulate with (EGF+Insulin) + Etoposide
numStocCells = 100 # Number of stochastic cells to simulate
STIMligs = [3.3,0.0,0.0,0.0,0.0,0.0,1721.0] # EGF+Insulin
nmxlsfile = 'WithGF_Etop_24pl72pl72hrs_3_'
ttd_wGF = []

ts = 30
NSteps1 = th1*3600/ts
NSteps1 = int(NSteps1)
NSteps2 = th2*3600/ts
NSteps2 = int(NSteps2)
NSteps3 = th3*3600/ts
NSteps3 = int(NSteps3)

Vn = 1.75E-12
Vc = 5.25E-12
spdata0 = pd.read_csv('Species_Brep.txt',header=0,index_col=0,sep="\t")
spdata = np.double(spdata0.values[:,1])
spdata[np.argwhere(spdata <= 1e-6)] = 0.0
genedata0, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)

sys.path.insert(0, os.path.abspath(model_output_dir))
currentDirectory = os.getcwd()

startTime = datetime.now()
print(startTime)
for nn in range(numStocCells): 
    xoutS_all = np.zeros(shape=((NSteps1+NSteps2+NSteps3)+1,len(spdata)))
    xoutS_all[0,:] = spdata  
    xoutG_all = np.zeros(shape=((NSteps1+NSteps2+NSteps3)+1,len(genedata0)))
    xoutG_all[0,:] = genedata0
        
    model_module = importlib.import_module(model_name)
    model = model_module.getModel()
    solver = model.getSolver() # Create solver instance
    model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints
    # First, serum-starvation simulation
    for qq in range(NSteps1):
        genedata,AllGenesVec = gm(flagD,ts,xoutG_all[qq,:],xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                   kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
        for ii,kk in enumerate(kmRNAs):
            model.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
        model.setInitialStates(xoutS_all[qq,:])
        rdata = amici.runAmiciSimulation(model, solver)  # Run simulation
        xoutS_all[qq+1,:] = rdata['x'][-1,:]
        xoutG_all[qq+1,:] = genedata
        if rdata['x'][-1,103] < rdata['x'][-1,105]:
            print('Apoptosis happened')
            break
    
    # If cell does not die, stimulate and simulate with EGF+Ins
    lenSim = len(xoutS_all[~np.all(xoutS_all == 0, axis=1)])
    if lenSim>NSteps1:
        xoutS_all[NSteps1,155:162] = STIMligs # Stim with EGF+Ins
        for ww in range(NSteps1,NSteps1+NSteps2):
            genedata,AllGenesVec = gm(flagD,ts,xoutG_all[ww,:],xoutS_all[ww,:],Vn,Vc,kGin,kGac, \
                                       kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
            for ii,kk in enumerate(kmRNAs):
                model.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
            model.setInitialStates(xoutS_all[ww,:])
            rdata = amici.runAmiciSimulation(model, solver)
            xoutS_all[ww+1,:] = rdata['x'][-1,:]
            xoutG_all[ww+1,:] = genedata
            if rdata['x'][-1,103] < rdata['x'][-1,105]:
                print('Apoptosis happened')
                break
        
        # If the cell still did not die, stimulate/simulate with additional Etop
        lenSim = len(xoutS_all[~np.all(xoutS_all == 0, axis=1)])
        if lenSim>(NSteps1+NSteps2): # Etoposide = 100000.0
            model.setFixedParameterById('k316_3',100000.0) # set Etop value
            for yy in range(NSteps1+NSteps2,NSteps1+NSteps2+NSteps3):
                genedata,AllGenesVec = gm(flagD,ts,xoutG_all[yy,:],xoutS_all[yy,:],Vn,Vc,kGin,kGac, \
                                           kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
                for ii,kk in enumerate(kmRNAs):
                    model.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
                model.setInitialStates(xoutS_all[yy,:])
                rdata = amici.runAmiciSimulation(model, solver)
                xoutS_all[yy+1,:] = rdata['x'][-1,:]
                xoutG_all[yy+1,:] = genedata
                if rdata['x'][-1,103] < rdata['x'][-1,105]:
                    print('Apoptosis happened')
                    break
    xoutS_all = xoutS_all[~np.all(xoutS_all == 0, axis=1)]
    xoutG_all = xoutG_all[~np.all(xoutG_all == 0, axis=1)]
    condsSDF = pd.DataFrame(data=xoutS_all,columns=[ele for ele in model.getStateIds()]) 
    condsSDF.to_csv(nmxlsfile+'S_'+str(nn)+'.txt',sep="\t")    
    condsSDF = None
    condsGDF = pd.DataFrame(data=xoutG_all) 
    condsGDF.to_csv(nmxlsfile+'G_'+str(nn)+'.txt',sep="\t")    
    condsGDF = None
    
    aa = np.argwhere(xoutS_all[:,105]>100.0)
    if len(aa)>0:
        aa2 = ((aa[0])*30.0)/3600.0
        ttd_wGF.append(aa2[0])
    else:
        ttd_wGF.append(1000.0)
print(datetime.now())
condsTDF = pd.DataFrame(data=ttd_wGF) 
condsTDF.to_csv(nmxlsfile+'ttds.txt',sep="\t")    
condsTDF = None